In [2]:
from scripts.word_sparse_matrix import *
from scripts.load_dataset import *
from tqdm import tqdm
import tensorflow as tf
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
emsize=5

wordset=load_dataset("./dataset/word_n63.mat").reshape(-1,4*41)
uniq_words = sorted(set(load_dataset("./dataset/word_n63.mat").reshape(-1)))

f = open('./Untitled Folder/ces-d.txt','r')
ces_d = list(map(int,f.readline().split(', ')))
ces_d = np.array([ces_d]).transpose(1,0)

#embedding = np.array(list(map(float,open('./w2v_out/w2v_out_embsize-'+str(emsize)+'.txt','r').read().split(',')))).reshape(-1,emsize)

#uniq_word_embedding = {word : embedded for word, embedded in zip(uniq_words, embedding)}

In [4]:
import scipy

In [5]:
dim_score = np.array(list(scipy.io.loadmat('./dataset/temp_dim_rating.mat')['dat'].squeeze()))

In [27]:
tt = []
for i in range(wordset.shape[0]):
    for j in range(wordset.shape[1]):
        tt.append(uniq_word_embedding[wordset[i,j]])
wordset = np.array(tt)

In [28]:
wordset=np.array(wordset).reshape(-1,164,emsize)

In [10]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 160,emsize])
y = tf.placeholder(tf.float32, [None, 1]) 
keep_prob = tf.placeholder(tf.float32)
n_hidden=200

W = tf.Variable(tf.random_normal([n_hidden,1]))
b = tf.Variable(tf.random_normal([1]))

cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)    
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)         
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

outputs, states = tf.nn.dynamic_rnn(multi_cell, x, dtype=tf.float32)
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
logits = tf.matmul(outputs, W) + b
#logits = tf.nn.tanh(logits)

def R_squared(y, y_pred):
    residual = tf.reduce_sum(tf.square(tf.subtract(x=y, y=y_pred)))
    total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
    r2 = tf.subtract(1.0, tf.div(residual, total))
    return r2

R2 = R_squared(y[:,0], logits[:,0])

In [11]:
cost = tf.reduce_mean(tf.losses.mean_squared_error(labels=y, predictions=logits))

optimizer = tf.train.AdamOptimizer(1e-2).minimize(cost)

In [12]:
config = tf.ConfigProto(log_device_placement=False, allow_soft_placement=True)
config.gpu_options.allow_growth = True

sess = tf.Session(config=config)
graph = tf.get_default_graph()

In [13]:
tr_idx=[0, 1, 3, 4, 6, 7, 8, 9, 12, 13, 15, 20,
        21, 23, 24, 25, 26, 27, 28, 29, 30, 31,
        32, 34, 36, 37, 38, 42, 43, 45, 46, 47,
        49, 50, 51, 53, 54, 56, 59, 60, 61, 62]

te_idx=[2, 5, 10, 11, 14, 16, 17, 18, 19, 22, 33,
        35, 39, 40, 41, 44, 48, 52, 55, 57, 58]

tr_wordset = dim_score[tr_idx]
tr_ces_d = ces_d[tr_idx]
te_wordset = dim_score[te_idx]
te_ces_d = ces_d[te_idx]

In [ ]:
iter_num=10000

with tqdm(total = iter_num) as pbar:
    sess.run(tf.global_variables_initializer())
    for step in range(iter_num):
        
        # Training
        tr_loss, tr_r2, _ = sess.run([cost, R2, optimizer], {x: tr_wordset, y : tr_ces_d, keep_prob : 0.7 })
        te_loss, te_r2, = sess.run([cost, R2], {x: te_wordset, y : te_ces_d, keep_prob:1.})
        
        pbar.update(1)
        pbar.set_description("[ Step : "+str(step+1)+"]")
        pbar.set_postfix_str(" Train Loss : "+'%.4f' % (tr_loss) + " Train R_Sqaure : "+'%.4f' % (tr_r2) +  ' Test Loss : ' + '%.4f'%(te_loss) + ' R_Sqaure : ' + '%.4f'%(te_r2))

[ Step : 2750]:  28%|██▊       | 2750/10000 [11:56<35:04,  3.45it/s,  Train Loss : 0.1424 Train R_Sqaure : 0.9982 Test Loss : 88.3415 R_Sqaure : -0.4827]   

In [97]:
red = sess.run(logits, {x: te_wordset,y : te_ces_d, keep_prob:1.})

In [99]:
for tt in zip(red[:,0]*43,ces_d[:,0]):
    print(tt)

(-3.7058489, 11)
(6.8751211, 43)
(-11.591451, 24)
(16.433159, 15)
(22.388157, 5)
(-8.7307968, 24)
(16.491894, 0)
(-1.5547183, 0)
(-7.0858641, 33)
(-1.1795514, 6)
(1.3317552, 30)
(2.0193491, 7)
(20.740721, 11)
(-1.543364, 7)
(1.3274186, 5)
(13.748446, 23)
(-3.5737519, 5)
(-2.5582914, 8)
(-12.800781, 20)
(-7.7891049, 5)
(-11.929564, 8)
